# dtree01.ipynb & data03.parquet

used a decision tree to remove non news (ads or unrelated news to each company)

also used percentagens intervals in order to keep more news:

- if newsProb in [.0, .4[ - trash

- if newsProb in [.4, .6] - filter setences: keep only the ones with any alias

- if newsProb in ].6, 1.] - keep everything

In [ ]:
import pandas as pd
import re
from joblib import load

In [ ]:
# FEATURES
# [['IstALIAS', 'propAN', 'txtSZ', 'countALI', 'countDTS', 'countHOUR', 'countCAPS']]

# added "aliases" to all func in order to run them all without worrying about the inputs

def IstALIAS(text, aliases):
    """where does the first alias appear, title?"""
    indexs = []
    for alias in aliases:
        index = text.lower().find(alias.lower())
        if index != -1:
            indexs.append(index)
    try:
        a = text[:min(indexs)].count(' ')
    except:
        a = 10000000000000000000
    return a

def propAN(text, aliases):
    """proportion of alphanumeric chars in the text"""
    alphanumeric_chars = sum(char.isalnum() for char in text)
    proportion = alphanumeric_chars / len(text)
    return proportion

def txtSZ(text, aliases):
    """text size"""
    return len(text)

def countALI(text, aliases):
    """count how many aliases appear in the text"""
    alias_count = {expression: 0 for expression in aliases}
    for alias in aliases:
        # Use re.escape to handle any special characters in the expression
        pattern = re.escape(alias.lower())
        matches = re.findall(alias, text.lower())
        alias_count[alias] = len(matches)
    return sum(alias_count.values())

def countDTS(text, aliases):
    """count how many dates appear in the text"""
    date_pattern = r'\b(\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{4}[-/]\d{1,2}[-/]\d{1,2})\b'
    # 10/11/2024', '10/10/2024', '12-25-1990', '2024-11-05', '01/10/2024'
    dates = re.findall(date_pattern, text)
    date_count = len(dates)
    return date_count

def countHOUR(text, aliases):
    """count how many hours (ex.: hh:mm) appear in the text"""
    time_pattern = r'\b([01]?[0-9]|2[0-3]):[0-5][0-9]\b'   
    occurrences = re.findall(time_pattern, text)
    return len(occurrences)

def countCAPS(text, aliases):
    """count how many WORDS are upper"""
    words = text.split()
    uppercase_word_count = sum(1 for word in words if word.isupper())
    return uppercase_word_count

# Load the saved model
clf = load('dtree01.joblib')

In [ ]:
def filter_sentences_by_keywords(text, aliases):
    # Split the text by punctuation and also by multiple spaces or newlines
    sentences = re.split(r'(?<=[.!?]) +|\s{2,}|\n+', text)
    # Filter sentences that contain any of the aliases
    filtered_sentences = [sentence for sentence in sentences if any(keyword.lower() in sentence.lower() for keyword in aliases)]
    # Join the filtered sentences back into a single string
    filtered_text = ' '.join(filtered_sentences)
    return filtered_text



features = ['IstALIAS', 'propAN', 'txtSZ', 'countALI', 'countDTS', 'countHOUR', 'countCAPS']

data = pd.read_parquet("data02.parquet")

sonae = data.iloc[3,0].copy()
data.iloc[3,0] = ["Sonae"] ############## ele vai buscar SON mesmo q seja son...

probs_distribution = [] #### probs_distri

for row in data.index:
    print(f"\n {row}", end = ": ")
    aliases = data.loc[row, "aliases"]
    for column in data.columns[1:]:
        print(column, end = " | ")
        validation = []
        for req in data.loc[row, column]:
            text = req["ExtractedText"]
            df = {}
            for feature in features:
                df[feature] = [globals()[feature](text, aliases)]
            #prediction = clf.predict(pd.DataFrame(df)) # binario
            probability = clf.predict_proba(pd.DataFrame(df))[0, 1]
            probs_distribution.append(round(probability, 3)) #### probs_distri
            if probability < 0.4:
                pass
            elif probability >= 0.4 and probability <= 0.6:
                req["newsProbability"] = round(probability, 3)
                req["ExtractedText"] = filter_sentences_by_keywords(text, aliases)
                validation.append(req)
            elif probability > 0.6:
                req["newsProbability"] = round(probability, 3)
                validation.append(req)
        data.loc[row, column] = validation

data.iloc[3,0] = sonae

data.to_parquet("data03.parquet")

In [ ]:
pd.read_parquet("data03.parquet").map(lambda x: len(x)) - pd.read_parquet("data02.parquet").map(lambda x: len(x))

In [ ]:
pd.read_parquet("data03.parquet").map(lambda x: len(x))

In [ ]:
data.iloc[0,2][0]

---

# data04.paquet

also:

- ~~save all data in a single column or something different inside the parquet~~

- ~~add news source to dicionary~~

- ~~convert timestamp to YYYYMM because ts is from archived, not the new itself~~

- ~~reorder dicionary~~

- ~~remove near duplicates (90%)~~

**merged api columns into news**

In [101]:
df = pd.read_parquet("data03.parquet").map(lambda x: list(x))
df['news'] = df.iloc[:, 1:].sum(axis=1)
df_filtered = df.iloc[:, [0, -1]]
df_filtered

,aliases,news
companies,,
Banco Comercial Português,"[Banco Comercial Português, BCP]",[{'ExtractedText': 'DN 13 de Setembro de 200...
Galp Energia,"[Galp Energia, GALP]",[{'ExtractedText': 'RTP Galp reforça posição n...
EDP,"[EDP, Energias de Portugal, Electricidade de P...",[{'ExtractedText': 'DN-Sinteses Negocios 9 de ...
Sonae,"[Sonae, SON]",[{'ExtractedText': 'DN-Sinteses 5 de Março de ...
Mota-Engil,"[Mota-Engil, EGL]",[{'ExtractedText': 'RTP Lucro da Mota-Engil so...


In [102]:
df_filtered.map(len)

,aliases,news
companies,,
Banco Comercial Português,2,1284
Galp Energia,2,1338
EDP,3,1456
Sonae,2,899
Mota-Engil,2,459


**convert timestamp +  add sources + reorder dictionary**

In [103]:
def tstampANDsourceANDreorderdict(lista):
    new_list = []
    noticias = pd.read_csv("noticias.csv")
    for req in lista:
        # news source
        linkToArchive = req["linkToArchive"]
        foundSource = False
        for index, row in noticias.iterrows():
            if row.iloc[0] in linkToArchive:
                req["newsSource"] = row.iloc[1]
                foundSource = True
                break
            else:
                pass
        if not foundSource:
            req["newsSource"] = "unknown"
        # timestamp
        req["tstamp"] = req["tstamp"][:6]
        # reorder dict
        ordered_keys = ["tstamp", "newsSource", "newsProbability", "linkToArchive", "ExtractedText"]
        reordered_dict = {key: req[key] for key in ordered_keys}
        # SAVE
        new_list.append(reordered_dict)
    return new_list

df_filtered.loc[:, "news"] = df_filtered["news"].map(lambda x: tstampANDsourceANDreorderdict(x))

In [104]:
df_filtered.iloc[0,-1][0]

{'tstamp': '200109',
 'newsSource': 'Diário de Notícias',
 'newsProbability': 0.933,
 'linkToArchive': 'https://arquivo.pt/wayback/20010913052557/http://www.dn.pt/int/13p4x.htm',
 'ExtractedText': 'DN \xa0 13 de Setembro de 2001 Intalações do BCP servem de refúgio As instalações do Banco Comercial Português, em Wall Street, foram utilizadas como refúgio por dezenas de pessoas depois dos atentados ao World Trade Center. Nenhum dos 48 funcionários daquela sucursal bancário ficou ferido e os dois trabalhadores que ainda não tinham comunicado com empresa fizeram-no na madrugada de ontem. Voo TAP para Newark chega hoje O voo 1331 da TAP que partiu terça-feira de Lisboa para Newark, nos Estados Unidos, deve chegar hoje ao seu destino. Segundo anunciou a porta-voz da companhia aérea, o avião foi direccionado para a cidade de Halifax, no Canadá, quando os EUA encerraram todos os seus aeroportos a vôos comerciais que se dirigissem para o território. EUA agradecem apoio português O representante

In [106]:
df_filtered.map(len)

,aliases,news
companies,,
Banco Comercial Português,2,1284
Galp Energia,2,1338
EDP,3,1456
Sonae,2,899
Mota-Engil,2,459


In [107]:
from fuzzywuzzy import fuzz
import random

def nearDuplicates(lista, threshold=90):
    total_data = len(lista) # status
    curr_data = 0 # status
    new_list = [lista[0]]
    texts = [lista[0]["ExtractedText"]]
    for req in lista[1:]:
        curr_data += 1 # status
        ExtractedText = req["ExtractedText"]
        similarity = 0
        for txt in texts:
            similarity = max(similarity, fuzz.ratio(txt, ExtractedText))
            if similarity > threshold:
                break
        if similarity <= threshold:
            new_list.append(req)
            texts.append(ExtractedText)
        if random.uniform(0, 1) < 0.01: # status
            print(f"{curr_data} of {total_data}", end = " | ") # status
    print("") # status
    return new_list

df_filtered.loc[:, "news"] = df_filtered["news"].map(lambda x: nearDuplicates(x))
df_filtered.map(len)

42 of 1284 | 309 of 1284 | 348 of 1284 | 799 of 1284 | 936 of 1284 | 1027 of 1284 | 1145 of 1284 | 1226 of 1284 | 
267 of 1338 | 315 of 1338 | 323 of 1338 | 339 of 1338 | 380 of 1338 | 386 of 1338 | 389 of 1338 | 412 of 1338 | 510 of 1338 | 570 of 1338 | 615 of 1338 | 764 of 1338 | 917 of 1338 | 950 of 1338 | 977 of 1338 | 1058 of 1338 | 1062 of 1338 | 1073 of 1338 | 1206 of 1338 | 
51 of 1456 | 76 of 1456 | 526 of 1456 | 660 of 1456 | 678 of 1456 | 689 of 1456 | 885 of 1456 | 993 of 1456 | 1203 of 1456 | 
209 of 899 | 285 of 899 | 598 of 899 | 623 of 899 | 715 of 899 | 734 of 899 | 758 of 899 | 779 of 899 | 873 of 899 | 890 of 899 | 
37 of 459 | 38 of 459 | 40 of 459 | 261 of 459 | 284 of 459 | 313 of 459 | 389 of 459 | 


,aliases,news
companies,,
Banco Comercial Português,2,807
Galp Energia,2,809
EDP,3,954
Sonae,2,520
Mota-Engil,2,296


In [ ]:
df_filtered.iloc[0,1][0]

In [110]:
#df_filtered.to_parquet("data04.parquet")

In [114]:
pd.read_parquet("data04.parquet")

,aliases,news
companies,,
Banco Comercial Português,"[Banco Comercial Português, BCP]",[{'ExtractedText': 'DN 13 de Setembro de 200...
Galp Energia,"[Galp Energia, GALP]",[{'ExtractedText': 'RTP Galp reforça posição n...
EDP,"[EDP, Energias de Portugal, Electricidade de P...",[{'ExtractedText': 'DN-Sinteses Negocios 9 de ...
Sonae,"[Sonae, SON]",[{'ExtractedText': 'DN-Sinteses 5 de Março de ...
Mota-Engil,"[Mota-Engil, EGL]",[{'ExtractedText': 'RTP Lucro da Mota-Engil so...


In [115]:
pd.read_parquet("data04.parquet").iloc[0,1][0]

{'ExtractedText': 'DN \xa0 13 de Setembro de 2001 Intalações do BCP servem de refúgio As instalações do Banco Comercial Português, em Wall Street, foram utilizadas como refúgio por dezenas de pessoas depois dos atentados ao World Trade Center. Nenhum dos 48 funcionários daquela sucursal bancário ficou ferido e os dois trabalhadores que ainda não tinham comunicado com empresa fizeram-no na madrugada de ontem. Voo TAP para Newark chega hoje O voo 1331 da TAP que partiu terça-feira de Lisboa para Newark, nos Estados Unidos, deve chegar hoje ao seu destino. Segundo anunciou a porta-voz da companhia aérea, o avião foi direccionado para a cidade de Halifax, no Canadá, quando os EUA encerraram todos os seus aeroportos a vôos comerciais que se dirigissem para o território. EUA agradecem apoio português O representante diplomático dos EUA em Portugal, William McGlynn, agradeceu a solidariedade com que o Governo e o povo têm acompanhado a tragédia. "Rezamos para que não existam vítimas portugues